# F-statistic Calibration
## Introduction
Distinguishing flat dose-response curves (i.e., completly inactive/active) is essential prior perfroming any cruve fitting. If flat curves are not distinguished from the true sigmoidal curves, it can lead to force fitting of sigmoidal models, which may produce low residual sum of square and be mathematically sound, but miss the messiness of real biological experiemnts, assay noise and biological meanings. Thus, it would be essential to statistically examine wether expeiremntal dose-response curves are signtifacntly different from a flat-line.

To address this issue, Bayer et al. (2023)[1], used simulations to re-calibrate the F-statistics for non-linear sigmoidal dose-response curves. Here, we adopted a similar appraoch to simulate null model and recalibrate F-statistics.

## Implementation
- **Models**:
    - 1-parameter model: The flat model assumes no dose effects and it would be a constant across all doses.
    - Two-parameter logistic model: in this model, given that we are using normalized dose-response cruves clipped between 0 and 1, we fixed the front and back plateau at 0 and 1, and kept the slope (B) and midpoint (C) to be adjustable.
- **Residual sum of sqaures (RSS)**: to measure how much the model's prediction miss the actual data points, we calculate the RSS as follows:
    - $\text{RSS} = \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$; where $ y_i $ is the observed (actual) data point for the $ i $-th observation, the $ \hat{y}_i $: is the predicted value from the model for the $ i $-th observation and $ n $: is the number of doses points (10; 128-0.25 µg/mL).
- **F-statistics**: to see if the more complex model (2-PL) exaplins the data better, we calculated the F-statistics as follows:
    - $F = \left( \frac{\text{RSS}_0 - \text{RSS}_1}{\text{RSS}_1} \right) \times (n - k)$; where $ \text{RSS}_0 $ is the RSS of the 1-parameter model, $ \text {RSS}_1 $ is the RSS of the 2-PL model, $ n $ number of doses and $ k $ is the number of parameters in the 2-PL models which is equal to 2.
        - Note: if the 2-PL fit fails or produces a higher RSS, it defaults to RSS0 to avoid invalid F values.
- **Curve-Fitting**: to find the best fit and prevent local minima, the 2-PL model is fitted using up to 7 different starting points to minimize the residual sum of squares (RSS) using the Trust Region Reflective (TRF) method and further minimzed using Nelder-Mead method.
    - The initial starting point is B = 1 and C = median concentration dose (i.e., 6).
    - For the remaining starting points, we use normally picked values from the bounds : B [-10, 10] and [Lowest Concentration/4 ,Highest Concentration × 4]
    - For each start, we use "scipy.optimize.curve_fit" with the TRF method, alllowing up to 8000 function evaluation.
    - The best parameters then will be further refined using scipy.optimize.minimize using Nelder-Mead method up to 20,000 evaluations.
        - Note: if fits was not succeeds, the simulation deafults to flat RSS.
- **Monte Carlo Simulation**: to calibrate the F-statistic under the null hypothesis (flat-curve), we perform 100,000 simulations per control type (Growth or Sterility) per organism.
    - Control Mean and SD Pools: we computed the mean and standard deviation of each growth and sterilty control samples, while discarding those with SD = 0 or missing data.
    - Simulation: using the pool of  mean and SD, we computed the simulated response data for each of 10 doses as follows:
        - $y_i = \mu + \epsilon_i, \quad \epsilon_i \sim \mathcal{N}(0, \sigma)$; where $ y_i $ represents the response at dose $ i $ (for $ i = 1, \dots, 10 $), $ \mu $ represents sampled mean from control data, $ \epsilon_i $: is random noise from a normal distribution with mean 0 and standard deviation $ \sigma $ (sampled from control data) and $ \mathcal{N}(0, \sigma) $ represent the normal distribution with mean 0 and standard deviation $ \sigma $
    - Subsequently, the RSS for flat and 2-PL models are obtained and F statistics is calcauted.
- **Distribution Fitting for Calibration:**: following collection of simulated F values:
    - Filtered the finite F values and compute a density histogram with bin size of 600
    - Used scipy.optimize.least_squares to minimze the residual between the histogram and the scaled F probability density function, where paramters are df1 (starting ~2), df2 (>1), loc, and scale.
- **Outputs**: results are stored in a dictionary by organism and control, including the fitted parameters, and pool sizes.

## References
[1] Florian P. Bayer, Manuel Gander, Bernhard Kuster & Matthew The (2023). CurveCurator: a recalibrated F-statistic to assess, classify, and explore significance of dose–response curves. *Nature Communications*, 14, Article 7902. https://doi.org/10.1038/s41467-023-43696-z

In [ ]:
"""
simulate_null_f_per_org.py – v8.8  (flat vs 2‑PL only + rich runtime metadata)

DescriptionÅ
-----------
Null model:
    flat (k = 1)  vs  2‑parameter logistic (k = 2, A = 0, D = 1)

• Performs Monte‑Carlo calibration for the F‑test family "flat_vs_2pl".
• Records df₁*, df₂*, loc, scale per control class (Growth, Sterility).
• Writes results to `null_f_params_full.json`.
• Captures runtime metadata (wall‑clock time, CPU usage, RSS, etc.)
  under a top‑level "_meta" key in the same JSON file.
"""

# ──────────────────────────────────────────────────────────────────────
# 0 ▸ imports & constants
# ──────────────────────────────────────────────────────────────────────
import os, json, math, warnings, multiprocessing as mp
import time, datetime, resource
from pathlib import Path
import re

import numpy as np
import pandas as pd
from   scipy.optimize import curve_fit, minimize, least_squares
from   scipy.stats    import f as f_dist
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from datetime import timezone

warnings.filterwarnings("ignore", category=RuntimeWarning)

CSV        = "/projects/amp/asalehi/Dose/data/clean/full_clean_hts.csv"
CTRL_NAMES = {"growth": "Growth Control",
              "sterility": "Sterility Control"}

ABS_ORDER  = ["128","64","32","16","8","4","2","1","0.5","0.25"]
IF_COLS    = [f"i{c}_win" for c in ABS_ORDER]

CONC    = np.array([128,64,32,16,8,4,2,1,0.5,0.25], float)
N_DOSES = CONC.size

N_SIM_NULL   = 100_000                 # simulations *per* control
TOTAL_CPUS   = mp.cpu_count()
N_WORKERS    = min(128, TOTAL_CPUS, N_SIM_NULL)

# ──────────────────────────────────────────────────────────────────────
# 1 ▸ helper functions
# ──────────────────────────────────────────────────────────────────────
flat  = lambda x, c: np.full_like(x, c)
_log2 = lambda z: np.log2(np.maximum(z, 1e-12))

def two_pl(x, B, C):                    # A = 0, D = 1
    return 1.0 / (1 + 2**(B * (_log2(C) - _log2(x))))

rss  = lambda y, yhat: np.sum((y - yhat) ** 2)
f_cc = lambda r0, r1, n, k: ((r0 - r1) / r1) * (n - k)  # df_num=1 implicitly

def fit_best(func, x, y, p0, bounds, n_start=6, *, rng=None):
    rng = rng or np.random.default_rng()
    best_rss, best_theta = math.inf, None
    lo, hi = bounds
    starts = [p0] + [rng.uniform(lo, hi) for _ in range(n_start)]

    for start in starts:
        try:
            theta, _ = curve_fit(func, x, y, p0=start, bounds=bounds,
                                 method="trf", max_nfev=8000)
            r = rss(y, func(x, *theta))
            if r < best_rss:
                best_rss, best_theta = r, theta
        except Exception:
            pass

    if best_theta is not None:
        res = minimize(lambda t: rss(y, func(x, *t)), best_theta,
                       method="Nelder-Mead",
                       options={"maxfev": 20_000, "xatol": 1e-8, "fatol": 1e-8})
        if res.success and rss(y, func(x, *res.x)) < best_rss:
            best_theta, best_rss = res.x, rss(y, func(x, *res.x))
    return best_theta, best_rss

def df_effective(F_vals, k):
    """four‑parameter non‑central F fit"""
    F_vals = F_vals[np.isfinite(F_vals)]
    if F_vals.size == 0:
        raise RuntimeError("All simulated F values were NaN/Inf.")

    hist, edges = np.histogram(F_vals, bins=600, density=True)
    centres     = 0.5 * (edges[1:] + edges[:-1])

    def resid(p):
        df1, df2, loc, scale = p
        z = (centres - loc) / scale
        return f_dist.pdf(z, df1, df2) / scale - hist

    p0 = [k, max(k, 2), 0.1, 1.0]
    lb = [2, 1, -np.inf, 0.1]
    ub = [1e6, 1e6,  np.inf, 10]
    sol = least_squares(resid, p0, bounds=(lb, ub), method="trf")
    return sol.x  # df1, df2, loc, scale

# ──────────────────────────────────────────────────────────────────────
# 2 ▸ Monte‑Carlo simulation  (single F‑family)
# ──────────────────────────────────────────────────────────────────────
def simulate_nulls(seed, n_sim, sigma_pool, mean_pool):
    rng = np.random.default_rng(seed)
    n_pools = len(sigma_pool)
    indices = rng.integers(0, n_pools, size=n_sim)

    F_two = np.empty(n_sim, np.float32)   # flat vs 2‑PL

    b_two = ([-10, CONC.min() / 4], [10, CONC.max() * 4])
    p0_two = [1.0, np.median(CONC)]

    for i in range(n_sim):
        idx = indices[i]
        base = mean_pool[idx]
        sd = sigma_pool[idx]
        y  = base + rng.normal(0, sd, N_DOSES)
        r0 = rss(y, flat(CONC, y.mean()))
        _, r2 = fit_best(two_pl, CONC, y, p0_two, b_two, rng=rng)
        r2 = r2 if r2 is not None else r0
        F_two[i] = f_cc(r0, r2, N_DOSES, 2)

    return F_two

# ──────────────────────────────────────────────────────────────────────
# 3 ▸ calibration wrapper
# ──────────────────────────────────────────────────────────────────────
def run_calibration(df_all):
    organisms = sorted(df_all['Organism'].unique())
    results = {}
    for org in organisms:
        print(f"Processing organism: {org}")
        df_org = df_all.query("Organism == @org")
        if df_org.empty:
            print(f"No data for {org}, skipping.")
            continue

        pools = {}
        for lbl, pid in CTRL_NAMES.items():
            ctrl_df = df_org.loc[df_org.Peptide_ID.eq(pid), IF_COLS].dropna(how='all')
            if ctrl_df.empty:
                continue
            means = ctrl_df.apply(lambda r: np.nanmean(r), axis=1).to_numpy(float)
            sigs = ctrl_df.apply(lambda r: np.nanstd(r, ddof=0), axis=1).to_numpy(float)
            mask = sigs > 0
            if np.sum(mask) >= 3:
                pools[lbl] = {"mean_pool": means[mask], "sigma_pool": sigs[mask]}

        if not pools:
            print(f"No valid controls for {org}, skipping.")
            continue

        # Generate sanity check plots
        n_controls = len(pools)
        if n_controls > 0:
            org_safe = re.sub(r'[^a-zA-Z0-9_-]', '_', org)
            fig, axs = plt.subplots(2, n_controls, figsize=(7 * n_controls, 8), dpi=300, sharey='row')
            if n_controls == 1:
                axs = axs[:, np.newaxis]

            color_means = "#408cbf"
            color_sds = "#ff9332"

            col_idx = 0
            for lbl in sorted(pools.keys()):
                mean_data = pools[lbl]["mean_pool"]
                sigma_data = pools[lbl]["sigma_pool"]
                axs[0, col_idx].hist(mean_data, bins=20, color=color_means, alpha=0.85, edgecolor="none")
                axs[0, col_idx].set_title(f'{lbl.capitalize()} Means', fontsize=12, fontweight='bold')
                axs[0, col_idx].set_xlabel('Mean Values', fontsize=10, fontweight='bold', labelpad=8)
                axs[0, col_idx].set_ylabel('Frequency', fontsize=10, fontweight='bold', labelpad=8)
                axs[1, col_idx].hist(sigma_data, bins=20, color=color_sds, alpha=0.85, edgecolor="none")
                axs[1, col_idx].set_title(f'{lbl.capitalize()} SDs', fontsize=12, fontweight='bold')
                axs[1, col_idx].set_xlabel('SD Values', fontsize=10, fontweight='bold', labelpad=8)
                axs[1, col_idx].set_ylabel('Frequency', fontsize=10, fontweight='bold', labelpad=8)
                col_idx += 1

            for ax in axs.flatten():
                ax.set_facecolor("white")
                ax.spines["top"].set_visible(False)
                ax.spines["right"].set_visible(False)
                ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, p: f"{x:.2g}" if x % 1 != 0 else f"{x:.0f}"))
                plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

            fig.suptitle(f"Control Pools Distributions for {org}", fontweight="bold", fontsize=14, y=0.98)
            fig.subplots_adjust(left=0.1, right=0.95, top=0.92, bottom=0.15, wspace=0.25, hspace=0.3)
            plt.tight_layout(rect=[0, 0.03, 1, 0.95], pad=2.0)
            plt.savefig(f'control_pools_hist_{org_safe}.png', dpi=300)
            plt.close(fig)
            print(f"✓ saved control_pools_hist_{org_safe}.png")

        results[org] = {}
        ctx = mp.get_context("fork")
        base, rem = divmod(N_SIM_NULL, N_WORKERS)

        for lbl, pool in pools.items():
            jobs = [
                (137 + 41*i,
                 base + (1 if i < rem else 0),
                 pool["sigma_pool"],
                 pool["mean_pool"])
                for i in range(N_WORKERS)
            ]
            with ctx.Pool(N_WORKERS) as mp_pool:
                outs = mp_pool.starmap(simulate_nulls, jobs)

            F_TWO = np.concatenate(outs)

            results[org][lbl] = {
                "flat_vs_2pl": dict(
                    zip(["df1", "df2", "loc", "scale"],
                        df_effective(F_TWO, 2))
                ),
                "sigma_pool_len": len(pool["sigma_pool"]),
            }

    return results

# ──────────────────────────────────────────────────────────────────────
# 4 ▸ main
# ──────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    mp.set_start_method("fork")
    print(f"→ using {N_WORKERS} workers")

    # ── timers & CPU usage snapshot ───────────────────────────────
    t0 = time.perf_counter()
    ru0_self = resource.getrusage(resource.RUSAGE_SELF)
    ru0_children = resource.getrusage(resource.RUSAGE_CHILDREN)

    # ── run main calibration ─────────────────────────────────────
    df_all = pd.read_csv(CSV)
    results = run_calibration(df_all)

    # ── aggregate CPU & time usage ───────────────────────────────
    wall_clock = time.perf_counter() - t0
    ru1_self = resource.getrusage(resource.RUSAGE_SELF)
    ru1_children = resource.getrusage(resource.RUSAGE_CHILDREN)

    user_cpu_sec = (ru1_self.ru_utime  - ru0_self.ru_utime)  + \
                   (ru1_children.ru_utime - ru0_children.ru_utime)
    sys_cpu_sec  = (ru1_self.ru_stime  - ru0_self.ru_stime)  + \
                   (ru1_children.ru_stime - ru0_children.ru_stime)
    max_rss_kb   = max(ru1_self.ru_maxrss, ru1_children.ru_maxrss)

    # ── attach metadata & write JSON ─────────────────────────────
    results["_meta"] = {
        "timestamp": datetime.datetime.now(timezone.utc).replace(microsecond=0).isoformat() + "Z",
        "script_version": "8.8",
        "n_simulations_per_ctrl": N_SIM_NULL,
        "n_workers": N_WORKERS,
        "cpu_count": TOTAL_CPUS,
        "wall_clock_sec": round(wall_clock, 3),
        "user_cpu_sec":   round(user_cpu_sec, 3),
        "sys_cpu_sec":    round(sys_cpu_sec, 3),
        "max_rss_kb":     int(max_rss_kb),
        "max_rss_mb":     round(max_rss_kb / 1024, 1)
    }

    Path("null_f_params_full.json").write_text(
        json.dumps(results, indent=2)
    )
    print("✓ wrote null_f_params_full.json with metadata")

## F-statistics Calibration Visual Diagnostics

To creat a diagnostic for the evaluation of the F-statistic calibaration, in the following I have simulated random data based the acquired parameters (df1, df2, loc, and scal) obtained from the calibration file to check if the simulated data behaves as expected under a null hypothesis (i.e., flat curve).To do so:

1. Pull parameters (df1, df2, loc, and scal) from the JSON for the flat vs2PL model.
2. Sample N number of F-values and p-values:
    - sample_F(params, n) function generates random F-values
    - sample_p(parms, n) functions samples F-values and computes p-values using the cumulative distribution function (CDF) of the F-distribution.
3. Generates plots:
    - QQ-Plot for F-values: here we comapre the theoretical quantiles (expected from F-distribution) vs. empirical (sorted simulated F-values). If the data points fall close to y=x, it suggests the simulated F follows the expected distribution.
    - QQ-Plot for p-values: here we compare uniform theoritical values (0 to 1) to sorted simulated p-values.
    - P-value Density Histogram: here we plot the histogram of p-values (excluding exacatly 1) with auto-bins as follows, if $ n \geq 2 $ and IQR > 0:
        - $\text{bins} = \left\lceil \frac{\max(x) - \min(x)}{2 \cdot \text{IQR} / n^{1/3}} \right\rceil$; where:
            - If $ n < 2 $, returns 10
            - If $ h $ is not finite or $ h \leq 0 $, return 50.
    - Empirical Cumulative Distribution Function (ECDF) of P-values: which plots the cumulative fraction of p-values vs. p-values to show how p-values accumulate and a uniform distribution indicate a good null calibration.

In [ ]:
"""
make_demo_from_calibration.py
====================================
Diagnostic plots for the single‑family calibration (flat vs 2‑PL).

"""
import json, warnings, re
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy.stats import f as f_dist, iqr

warnings.filterwarnings("ignore", category=RuntimeWarning)

CAL_JSON = "/projects/amp/asalehi/Dose/f_calibration/null_f_params_full_1M.json"
FIG_DIR  = "/projects/amp/asalehi/Dose/f_calibration/figures"
N_DEMO   = 2_000_000
MAX_BINS = 600

Path(FIG_DIR).mkdir(parents=True, exist_ok=True)

# ───────────────────────── colour palette ─────────────────────────
# khaki‑like tone requested: #c3b091
COLORS = {"flat_vs_2pl": "#c3b091"}

# ───────────────────────── helper functions ───────────────────────
def prettify(ax):
    ax.set_facecolor("white")
    ax.yaxis.grid(True, ls="-", lw=0.5, alpha=0.5, zorder=0)
    ax.xaxis.grid(True, ls="-", lw=0.5, alpha=0.5, zorder=0)
    ax.set_axisbelow(True)
    for spine in ("top", "right"):
        ax.spines[spine].set_visible(False)
    ax.xaxis.set_major_formatter(
        ticker.FuncFormatter(lambda x, p: f"{x:.2g}" if x % 1 else f"{x:.0f}")
    )
    plt.setp(ax.get_xticklabels(), rotation=0, ha="center")

def sample_F(params, n=N_DEMO):
    df1, df2, loc, scale = params.values()
    return loc + scale * f_dist.rvs(df1, df2, size=n)

def sample_p(params, n=N_DEMO):
    df1, df2, loc, scale = params.values()
    F = loc + scale * f_dist.rvs(df1, df2, size=n)
    return 1.0 - f_dist.cdf((F - loc) / scale, df1, df2)

def freedman_bins(arr, *, max_bins=MAX_BINS):
    arr = arr[np.isfinite(arr)]
    if arr.size < 2:
        return 10
    h = 2 * iqr(arr) / np.cbrt(arr.size)
    if not np.isfinite(h) or h <= 0:
        return 50
    return max(10, min(int(np.ceil(np.ptp(arr) / h)), max_bins))

# ─────────────────────── load calibration ────────────────────────
cal = json.loads(Path(CAL_JSON).read_text())

# ───────────────────── per‑organism loop ─────────────────────────
for org in sorted(k for k in cal if k != "_meta"):
    print(f"Processing organism: {org}")
    ctrl_blocks = {k: v for k, v in cal[org].items() if k in ("growth", "sterility")}
    if not ctrl_blocks:
        print(f"No valid controls for {org}, skipping.")
        continue

    org_safe   = re.sub(r"[^a-zA-Z0-9_-]", "_", org)
    controls   = sorted(ctrl_blocks.keys())
    n_controls = len(controls)

    data = {}  # Initialize data dictionary for this organism

    # Sample demo data for each control 
    for ctrl in controls:
        params = ctrl_blocks[ctrl]["flat_vs_2pl"]
        data[ctrl] = {
            "df1":      params["df1"],
            "df2":      params["df2"],
            "loc":      params["loc"],
            "scale":    params["scale"],
            "F_vals":   sample_F(params),
            "p_vals":   sample_p(params),
            "fam_label":"flat vs 2pl",
        }
        data[ctrl]["p_sorted"] = np.sort(data[ctrl]["p_vals"])

    # Figure
    fig, axs = plt.subplots(
        2, n_controls * 2,
        figsize=(5 * n_controls * 2, 10),
        dpi=800, facecolor="white"
    )
    axs = np.atleast_2d(axs)  # ensure 2‑D

    for col_idx, ctrl in enumerate(controls):
        d = data[ctrl]
        F_vals, p_vals, p_sorted = d["F_vals"], d["p_vals"], d["p_sorted"]
        df1, df2, loc, scale     = d["df1"], d["df2"], d["loc"], d["scale"]
        fam_label = d["fam_label"]
        base = col_idx * 2  # first column for this control

        # QQ‑plot (F, log‑log)
        ax = axs[0, base]
        prettify(ax)
        n = len(F_vals)
        q_theo = scale * f_dist.ppf((np.arange(1, n + 1) - 0.5) / n, df1, df2) + loc
        q_emp  = np.sort(F_vals)
        ax.scatter(q_theo, q_emp, s=10, color=COLORS["flat_vs_2pl"], alpha=0.85)
        ax.set_xscale("log"); ax.set_yscale("log")
        lim = (min(q_emp.min(), q_theo.min()), max(q_emp.max(), q_theo.max()))
        ax.plot(lim, lim, "k--", lw=1)
        ax.set_xlim(lim); ax.set_ylim(lim)
        ax.set_xlabel("Theoretical quantiles (log₁₀)", fontsize=10, fontweight="bold")
        ax.set_ylabel("Empirical quantiles (log₁₀)",  fontsize=10, fontweight="bold")
        ax.set_title(f"{ctrl.title()} – QQ F", fontsize=12, fontweight="bold")

        # QQ‑plot (p)
        ax = axs[0, base + 1]
        prettify(ax)
        u_theo = (np.arange(1, len(p_sorted) + 1) - 0.5) / len(p_sorted)
        ax.scatter(u_theo, p_sorted, s=10, color=COLORS["flat_vs_2pl"], alpha=0.85)
        ax.plot([0, 1], [0, 1], "k--", lw=1)
        ax.set_xlim(0, 1); ax.set_ylim(0, 1)
        ax.set_xlabel("Theoretical U(0,1)", fontsize=10, fontweight="bold")
        ax.set_ylabel("Empirical p",        fontsize=10, fontweight="bold")
        ax.set_title(f"{ctrl.title()} – QQ p", fontsize=12, fontweight="bold")

        # p‑value density
        ax = axs[1, base]
        prettify(ax)
        bins = freedman_bins(p_vals[p_vals < 1])
        ax.hist(
            p_vals[p_vals < 1],
            bins=bins, density=True,
            color=COLORS["flat_vs_2pl"], alpha=0.85, edgecolor="none",
            label=fam_label
        )
        ax.axhline(1, ls="--", lw=1, color="grey", label="ideal null")
        ax.set_xlim(0, 1)
        ax.set_xlabel("p‑value", fontsize=10, fontweight="bold")
        ax.set_ylabel("Density",  fontsize=10, fontweight="bold")
        ax.set_title(f"{ctrl.title()} – p‑value density", fontsize=12, fontweight="bold")
        # legend with boxed, semi‑transparent background
        leg = ax.legend(loc="lower left",
                        bbox_to_anchor=(0.02, 0.02),  # <- lower corner inside panel
                        frameon=True,fontsize=8)
        leg.get_frame().set_facecolor("white")
        leg.get_frame().set_alpha(0.95)

        # ECDF of p‑values
        ax = axs[1, base + 1]
        prettify(ax)
        y_ecdf = np.linspace(1 / len(p_sorted), 1, len(p_sorted))
        ax.plot(p_sorted, y_ecdf, lw=5, color=COLORS["flat_vs_2pl"], label=fam_label)
        ax.plot([0, 1], [0, 1], "k--", lw=1, label="ideal")
        ax.set_xlim(0, 1); ax.set_ylim(0, 1)
        ax.set_xlabel("p‑value", fontsize=10, fontweight="bold")
        ax.set_ylabel("ECDF",    fontsize=10, fontweight="bold")
        ax.set_title(f"{ctrl.title()} – ECDF", fontsize=12, fontweight="bold")
        leg = ax.legend(loc="lower left",
                        bbox_to_anchor=(0.02, 0.02),  # <- lower corner inside panel
                        frameon=True, fontsize=8)
        leg.get_frame().set_facecolor("white")
        leg.get_frame().set_alpha(0.95)

    # Final styling & save
    fig.suptitle(
        f"{org} – Diagnostic plots (flat vs 2‑PL)",
        fontsize=14, fontweight="bold", y=0.98
    )
    fig.tight_layout(rect=[0, 0.03, 1, 0.95], pad=2.0)
    fig.savefig(Path(FIG_DIR, f"{org_safe}_diagnostics_2pl.png"))
    plt.close(fig)

    print(f"✓ generated combined facet plot for {org}")